In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon, shape
import geopandas as gpd
import pickle

# Import custom functions from `scripts` folder
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scripts.clean_tweets import geometrize_tweets, convert_shapefile_crs, find_frequencies
from scripts.home_location import assign_home_location
from scripts.analyze_buenos_aires import summary_stats, filter_and_home_assign

np.random.seed(42)
pd.set_option('display.max_columns', 999)

# Loading data

(If you have access to the pickled data files, begin running cells at the cell that reads `### PICKLE FILE ###`)

In [2]:
# Loading 2015 tweets
ba15 = pd.read_csv('../data/tweets/ba_2015.csv')

In [3]:
# Loading shapefiles
ba_shapefiles = convert_shapefile_crs(
    gpd.read_file('../data/shapefiles/buenos_aires_shapefiles/BA_2010.shp')
)

# Filtering data and adding tracts

First, we want to filter the data to exclude rare Twitter users (tweet count below median number of tweets/user) and power users (tweet count above 99th percentile of tweets/user). Methodology is as follows:

1. Compute 50th and 99th percentile of tweets/user
1. Select the subset of tweets made by users falling between those bounds
1. Geometrize tweet data by using lat/lon
1. Compute a spatial join between tweets and the tract shapefiles
1. Add datetime information for home location analysis
1. Assign home locations (`home` column) and whether or not tweet is made from home (`is_home` column)

In [ ]:
# Original dataset analysis
pct_50, pct_99 = summary_stats(ba15)

18962977 total tweets
333464 unique users

Median number of tweets/user: 6.0 tweets
99th percentile of tweets/user: 866.0 tweets


In [ ]:
ba15 = filter_and_home_assign(ba15, ba_shapefiles, pct_50, pct_99)

In [ ]:
### PICKLE FILE ###
# protocol=4 for large data files (https://stackoverflow.com/a/56151766)
with open('../data/ba15.pkl', 'wb') as file:
    pickle.dump(ba15, file, protocol=4)

# with open('../data/ba15.pkl', 'rb') as file:
#     ba15 = pickle.load(file)

# Analysis

Before we begin our analysis, it helps to show a visual analysis of where our tweets are located. In the map below, we differentiate between the tweets that are assigned a `codigo` in our spatial join:

In [ ]:
### Plot a 1% sample of 2015 tweets ###
# Green tweets: Joined to a codigo
# Orange tweets: No codigo assigned (NaN)
# Gray (barely visible beneath the blue): Buenos Aires shapefile

fig, ax = plt.subplots(figsize=(10, 7))
ba_shapefiles['geometry'].plot(ax=ax, color='gray')
sample = ba15.sample(frac=0.01, random_state=42)
sample[~sample['codigo'].isnull()].plot(ax=ax, marker='o', color='green', alpha=0.01, label='Green: Within Metropolitan Area')
sample[sample['codigo'].isnull()].plot(ax=ax, marker='o', color='orange', alpha=0.01, label='Orange: Outside Metropolitan Area')
plt.legend()
plt.axis([-58.6, -58.3, -34.75, -34.5])
plt.title('Buenos Aires tweets, 2015: Metropolitan vs. Outside');

From this, we see that we can use `[~ba15['codigo'].isnull()]` as a filter to select just the tweets in the metropolitan area, which is our area of interest. For all following analyses, we will restrict the population to just the tweets that fall inside the metropolitan area depicted by the shapefile.

Our first summary statistic computes the ratio of tweets that have a home assignment.

In [ ]:
num_home_assigned_tweets = (~ba15.loc[~ba15['codigo'].isnull(), 'home'].isnull()).sum()
n_tweets = len(ba15[~ba15['codigo'].isnull()])

print("{}/{} = {:.2%} of tweets have a home assignment.".format(
    num_home_assigned_tweets, n_tweets, num_home_assigned_tweets / n_tweets
))

Next, we are interested in the ratio of users that have a home assignment.

In [ ]:
num_users_with_home = (~ba15.loc[~ba15['codigo'].isnull()].groupby('u_id')['home'].first().isnull()).sum()
n_users = len(ba15[~ba15['codigo'].isnull()].groupby('u_id'))

print("{}/{} = {:.2%} of users have a home assignment.".format(
    num_users_with_home, n_users, num_users_with_home / n_users
))

Lastly, we want to know the ratio of tweets that were made from home (i.e. `codigo` and `home` are equal).

In [ ]:
num_tweets_from_home = ba15.loc[~ba15['codigo'].isnull(), 'is_home'].sum()

print("{}/{} = {:.2%} of tweets are posted from the home tract.".format(
    num_tweets_from_home, n_tweets, num_tweets_from_home / n_tweets
))

Our analysis concludes with a visual representation of this last information; namely, how are the tweets spread out across the entire metropolitan area of Buenos Aires with respect to residence/nonresidence?

In [ ]:
### Plot a 5% sample of 2015 tweets ###
# Blue tweets: Resident
# Red tweets: Non-resident
# Gray (barely visible beneath the blue): Buenos Aires shapefile

fig, ax = plt.subplots(figsize=(10, 7))
ba_shapefiles['geometry'].plot(ax=ax, color='gray')
sample = ba15[~ba15['codigo'].isnull()].sample(frac=0.05, random_state=42)
sample[~sample['is_home']].plot(ax=ax, marker='o', color='red', alpha=0.01, label='Red: Non-Resident')
sample[sample['is_home']].plot(ax=ax, marker='o', color='blue', alpha=0.01, label='Blue: Resident')
plt.legend()
plt.axis([-58.6, -58.3, -34.75, -34.5])
plt.title('Buenos Aires tweets, 2015: Resident vs. Non-resident');

# Saving output

For future reference, we save two outputs:

1. Number of tweets + home tweet ratio for each codigo
1. Tweets from the following focus codigos (with some restrictions, specified in code):
    - Parque Patricios
    - Boca

In [ ]:
##### 1. Number of tweets + home tweet ratio for each codigo #####
(
    ba15
    .loc[~ba15['codigo'].isnull(), ['codigo', 'is_home']]
    .groupby('codigo')
    .agg([len, np.mean])
    .T.reset_index(drop=True).T
    .rename(columns={0:'count', 1:'home_tweet_ratio'})
).to_csv('../data/ba15_by_codigo.csv')

In [ ]:
##### 2. Tweets from focus codigos #####
def save_focus_codigo_data(data, year):
    parque = data[(data['BARRIOS'] == 'PARQUE PATRICIOS') | 
                  (data['codigo'].isin([20041001, 20012905, 20032309]))]
    boca = data[(data['BARRIOS'] == 'BOCA') & (data['codigo'] != 20041801)]
    
    parque.to_csv('../data/ba' + str(year)[-2:] + '_PARQUE_PATRICIOS.csv', index=False)
    boca.to_csv('../data/ba' + str(year)[-2:] + '_BOCA.csv', index=False)
    
save_focus_codigo_data(ba15, 2015)